# ENCODE DATASETS

In [0]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
import os

from google.colab import drive
drive.mount('/content/drive')

In [0]:
dataset = '/content/drive/My Drive/data/gpt2/kaggle/articles-2_train.txt'

dataset_size = round((os.path.getsize(dataset) / 1024 / 1024), 2)
print('Size of dataset: {} MB'.format(dataset_size))

output_encoded_file = '{}_encoded.npz'.format(dataset.split('.')[0])
gpt2.download_gpt2(model_name='355M')
gpt2.encode_dataset(dataset, model_dir='models', out_path=output_encoded_file, model_name='355M')
output_encoded_file_size = round((os.path.getsize(output_encoded_file) / 1024 / 1024), 2)
print('Size of encoded output file: {} MB'.format(output_encoded_file_size))


# TRAIN MODEL

In [0]:
!pip install toposort
!git clone https://github.com/minimaxir/gpt-2-simple.git
!pip install loguru

from google.colab import drive
drive.mount('/content/drive')

!cp '/content/drive/My Drive/data/gpt2/gpt_2.py' gpt-2-simple/gpt_2_simple

In [0]:
!nvidia-smi

In [0]:
%cd gpt-2-simple
%tensorflow_version 1.x
import os

In [0]:
dataset = 'articles-2_train_encoded.npz'
source = dataset.split('_')[0]
model_name = '355M'
steps = 30000
restore_from = 'latest' # 'latest' or 'fresh'
run_name = f'{source}_A'
print_every = 200
sample_every = 1000
sample_num = 1
save_every = 1000
learning_rate = 1e-4
max_checkpoints = 1 # int(steps / save_every) + 1

checkpoints_file = f"/content/drive/My Drive/data/gpt2/kaggle/{source}_checkpoints.tar.gz"
train_file = f"/content/drive/My Drive/data/gpt2/kaggle/{dataset}"

if os.path.isfile(checkpoints_file):
  !tar -xzvf "{checkpoints_file}"

!cp "{train_file}" .
!ls -ltrh "{train_file}"

In [0]:
import gpt_2_simple as gpt2

gpt2.download_gpt2(model_name=model_name)

sess = gpt2.start_tf_sess()
gpt2.finetune(sess,
              dataset=train_file,
              model_name=model_name,
              steps=steps,
              restore_from=restore_from,
              run_name=run_name,
              print_every=print_every,
              sample_every=sample_every,
              sample_num=sample_num,
              save_every=save_every,
              learning_rate=learning_rate,
              max_checkpoints=max_checkpoints
              )

In [0]:
#### BIGGER MODEL
# %tensorflow_version 1.x
# !pip install -q gpt-2-simple
# import gpt_2_simple as gpt2

# model_name = "1558M"
# gpt2.download_gpt2(model_name=model_name)
# sess = gpt2.start_tf_sess()
# gpt2.load_gpt2(sess, model_name=model_name)

# LOAD PREV MODEL

In [0]:
!nvidia-smi

In [0]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
import os

from google.colab import drive
drive.mount('/content/drive')

In [0]:
run_name = 'articles-2_A'
!tar -xzvf '/content/drive/My Drive/data/gpt2/kaggle/articles-2_checkpoints.tar.gz'

In [0]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name=run_name) # checkpoint_dir='.'

In [0]:
for i in range(20):
  gen_file = '{}_{}_samples.txt'.format(run_name, i)
  gpt2.generate_to_file(sess,
                        # checkpoint_dir='.',
                        prefix='<|startoftext|>',
                        truncate='<|endoftext|>',
                        destination_path=gen_file,
                        run_name=run_name,
                        nsamples=50,
                        batch_size=10
                        )

In [0]:
category = run_name.split('-')[0]
for f in os.listdir('.'):
  if f.endswith('samples.txt'):
    !mv {f} '/content/drive/My Drive/data/gpt2/kaggle/samples/{category}'